In [12]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
#03-05-2015 06:00:05
#18-09-2015 05:59:48
'''qcat разобраться, использовать для разделения. Попытаться посчитать и R и F в рамках одной сортировки groupby'''

df_events = pd.read_csv('D:\Питон\Project X\Data sets\events.csv')

In [13]:
def rfm_score(df_events):
    df_with_uniq_id = (df_events[~df_events['transactionid'].isna()]    #убарает строки без покупок, фильтрует по времени,
                       .sort_values('timestamp', ascending=False)       #по убыванию, оставляет уникальные ID с последними покупками
                       .drop_duplicates('visitorid', keep='first'))
    step_for_r = int(df_with_uniq_id['visitorid'].count() * 0.2)        #20% шаг по кол-ву уникальных ID
    list_id = df_with_uniq_id['visitorid'].to_list()                    #список всех ID
    list_r = [r for r in range(5,0,-1) for l in range(step_for_r)]      #список оценок по давности покупки
    
    df_counts_by_id = (df_events[~df_events['transactionid']
                       .isna()].groupby('visitorid').count())            #кол-во событий у каждого ID выкинуть !выкинуть строки без покупок!
    top_num_of_trans = max(df_counts_by_id['transactionid'])            #Максимальное значение покупок
    step_for_f = top_num_of_trans * 0.2                                 #20% шаг по кол-ву покупок
    list_f = []                                                         #список оценок по кол-ву покупок
    for id in list_id:                                                  #цикл для выставления оценок
        num_trans = df_counts_by_id.loc[id]['transactionid']            #кол-во покупок по пользователю
        if top_num_of_trans >= num_trans > (top_num_of_trans - step_for_f):
             list_f.append(5)
        elif (top_num_of_trans - step_for_f) >= num_trans > (top_num_of_trans - step_for_f * 2):
             list_f.append(4)
        elif (top_num_of_trans - step_for_f * 2) >= num_trans > (top_num_of_trans - step_for_f * 3):
             list_f.append(3)
        elif (top_num_of_trans - step_for_f * 3) >= num_trans > (top_num_of_trans - step_for_f * 4):
             list_f.append(2)
        elif (top_num_of_trans - step_for_f * 4) >= num_trans >= 0:
             list_f.append(1)
    
    if len(list_r) < len(list_id):                                     #на случай если кол-во пользователей не кратно 5 и длинна list_r
        for i in range(len(list_id) - len(list_r)):                    #меньше длинны list_id
            list_r.append(1)
    
    return list_id, list_r, list_f

a = rfm_score(df_events)

In [14]:
df_rfm = pd.DataFrame({'VisitorID': a[0], 'R Score': a[1], 'F Score': a[2]})
df_rfm['F Score'].value_counts()

F Score
1    11707
2        9
4        1
5        1
3        1
Name: count, dtype: int64

In [11]:
df_events[~df_events['transactionid'].isna()].groupby('visitorid').count().sort_values('transactionid', ascending=False).info()

<class 'pandas.core.frame.DataFrame'>
Index: 11719 entries, 1150086 to 1407398
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   timestamp      11719 non-null  int64
 1   event          11719 non-null  int64
 2   itemid         11719 non-null  int64
 3   transactionid  11719 non-null  int64
dtypes: int64(4)
memory usage: 457.8 KB
